In [1]:
%load_ext autoreload
%autoreload 2

# Parquet validation

In [1]:
import pandas as pd
from pathlib import Path
import sys 

In [2]:
basepath: Path = Path('/N/project/openalex')
snapshot_dir = basepath / 'OpenAlex' / 'openalex-snapshot' / 'data'
processed_dir = basepath / 'ssikdar' / 'processed'
temp_dir = basepath / 'ssikdar' / 'temp'
csv_dir = basepath / 'ssikdar' / 'csv-files'

## Read parquet files

In [3]:
for parq_file in processed_dir.glob('*.h5'):
    df = pd.read_hdf(parq_file)
    display(parq_file.stem, f'{df.shape[0]:,}')

'authors'

'450,001'

'authors_ids'

'450,001'

'authors_counts_by_year'

'641,115'

In [5]:
df = pd.read_hdf(processed_dir / 'authors.h5')
len(df)

5884449

In [9]:
f'{len(df):,}'

'5,884,449'

## Figure out HDF stuff

In [49]:
del store

NameError: name 'store' is not defined

In [51]:
store = pd.HDFStore(processed_dir / 'authors.h5')

In [52]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: /N/project/openalex/ssikdar/processed/authors.h5

In [54]:
df = store.df

In [55]:
df[df.isna().any(axis=1)]

,id,orcid,display_name,display_name_alternatives,works_count,cited_by_count,last_known_institution,works_api_url,updated_date


In [43]:
store.root.df.table

/df (Group) ''
  children := []

In [9]:
stored_df = store.df

In [10]:
stored_df.dtypes

id                           object
orcid                        object
display_name                 object
display_name_alternatives    object
works_count                   int64
cited_by_count                int64
last_known_institution       object
works_api_url                object
updated_date                 object
dtype: object

In [11]:
def get_dtypes(path, **kwargs):
    sample = pd.read_csv(path, nrows=2000) # Load Sample datadtypes = sample.dtypes # Get the dtypes
    dtype_dictionary = {}
    dtypes = sample.dtypes # Get the dtypes
    parse_dates = []
    for col in sample.columns:
        if 'date' in col:  # turn dates into datetime objects
            parse_dates.append(col)
        if any(x in col for x in ('id', '_id')):  # id's turn into categories
            dtype_dictionary[col] = 'string'
        elif 'is_' in col:
            dtype_dictionary[col] = 'bool'
        elif sample[col].dtype.name == 'object':  # turn objects into strings
            dtype_dictionary[col] = 'string'
        elif sample[col].dtype.name == 'int64':  # use Int64 to have support for NaNs
            dtype_dictionary[col] = 'Int64'
        else:
            dtype_dictionary[col] = str(dtypes[col])  # Load Data with increased speed and reduced memory.
        if col in ['wikipedia', 'mag', 'twitter', 'scopus']:
            dtype_dictionary[col] = 'object'
    return dtype_dictionary

In [18]:
dtype = get_dtypes(csv_dir / 'authors_counts_by_year.csv.gz')
dtype

{'author_id': 'string',
 'year': 'Int64',
 'works_count': 'Int64',
 'cited_by_count': 'Int64'}

In [24]:
df = pd.read_csv(csv_dir / 'authors_ids.csv.gz', nrows=10000)

In [28]:
df.twitter.value_counts()

Series([], Name: twitter, dtype: int64)